# Bi-Directional LSTM

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df1 = pd.read_csv('./fake_job_postings.csv')

In [ ]:
df1.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [ ]:
df1.shape

(17880, 18)

In [ ]:
df=df1[['description','requirements','fraudulent']]
df.head()

,description,requirements,fraudulent
0,"Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,0
1,Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,0
2,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,0
3,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",0
4,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,0


In [ ]:
df=df.dropna()

In [ ]:
df.shape

(15185, 3)

In [ ]:
X=df.drop('fraudulent',axis=1)

In [ ]:
y=df['fraudulent']

In [ ]:
y.value_counts()

0    14473
1      712
Name: fraudulent, dtype: int64

In [ ]:
import tensorflow as tf
tf.__version__

'2.8.0'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout
from keras.layers import TimeDistributed

In [ ]:
voc_size=5000

In [ ]:
message = X.copy()

In [ ]:
message['description'][1]

"Organised - Focused - Vibrant - Awesome!Do you have a passion for customer service? Slick typing skills? Maybe Account Management? ...And think administration is cooler than a polar bear on a jetski? Then we need to hear you!\xa0We are the Cloud Video Production Service and opperating on a glodal level. Yeah, it's pretty cool. Serious about\xa0delivering a world class product and excellent customer service.Our rapidly expanding business is looking for a talented Project Manager to manage the successful delivery of video projects, manage client communications and drive the production process. Work with some of the coolest brands on the planet and learn from a global team that are representing NZ is a huge way!We are entering the next growth stage of our business and growing quickly internationally. \xa0Therefore, the position is bursting with opportunity for the right person entering the business at the right time.\xa090 Seconds, the worlds Cloud Video Production Service -\xa0http://90

In [ ]:
message.reset_index(inplace=True)

In [ ]:
!pip install ntlk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement ntlk (from versions: none)
ERROR: No matching distribution found for ntlk


In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(message)):
    review = re.sub('[^a-zA-Z]', ' ', message['description'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus[1]

'organis focus vibrant awesom passion custom servic slick type skill mayb account manag think administr cooler polar bear jetski need hear cloud video product servic opper glodal level yeah pretti cool seriou deliv world class product excel custom servic rapidli expand busi look talent project manag manag success deliveri video project manag client commun drive product process work coolest brand planet learn global team repres nz huge way enter next growth stage busi grow quickli intern therefor posit burst opportun right person enter busi right time second world cloud video product servic http url fbe afac cd c f b eef e e f ca dd second world cloud video product servic enabl brand agenc get high qualiti onlin video content shot produc anywher world fast afford manag seamlessli cloud purchas publish second remov hassl cost risk speed issu work regular video product compani manag everi aspect video project beauti onlin experi grow network rate video profession countri dedic product suc

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr[1]

[1391,
 3748,
 4992,
 3683,
 2875,
 2330,
 187,
 4762,
 4018,
 1029,
 3392,
 4041,
 891,
 790,
 1906,
 3457,
 3522,
 1246,
 2805,
 2944,
 1149,
 2245,
 37,
 3908,
 187,
 3435,
 2450,
 1120,
 1647,
 1107,
 637,
 3947,
 1435,
 3275,
 1711,
 3908,
 3305,
 2330,
 187,
 4814,
 2312,
 1698,
 3100,
 4715,
 4089,
 891,
 891,
 225,
 4,
 37,
 4089,
 891,
 1600,
 1983,
 2629,
 3908,
 620,
 4081,
 590,
 4394,
 3986,
 1553,
 2329,
 2383,
 1062,
 3523,
 2869,
 2811,
 2711,
 4514,
 4448,
 1705,
 1698,
 4649,
 3173,
 2267,
 1170,
 1010,
 2766,
 2658,
 3366,
 1207,
 2711,
 1698,
 3366,
 1121,
 3480,
 3275,
 2245,
 37,
 3908,
 187,
 3512,
 2641,
 1035,
 3791,
 2458,
 2961,
 4913,
 1225,
 2491,
 4063,
 4063,
 4913,
 4867,
 2397,
 3480,
 3275,
 2245,
 37,
 3908,
 187,
 4680,
 4394,
 1010,
 2300,
 1538,
 4123,
 3729,
 37,
 2213,
 2567,
 3119,
 38,
 3275,
 2571,
 4304,
 891,
 965,
 2245,
 989,
 3231,
 3480,
 4573,
 257,
 4255,
 4732,
 4040,
 440,
 4081,
 145,
 37,
 3908,
 2353,
 891,
 1194,
 1924,
 37,
 408

In [ ]:
sent_length=40
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[4355 2213  891 ... 2809 3879 4650]
 [ 225 4048 3320 ... 4174 1945 3182]
 [   0    0    0 ... 4019 3862 2531]
 ...
 [3481 2508  976 ... 4411 4475 1882]
 [4631  694 2522 ... 4185 2267 4089]
 [1118 1546  654 ... 4000 4173 1460]]


In [ ]:
embedded_docs[0]

array([4355, 2213,  891, 1763, 3179, 4000, 2951, 4864,  134, 2788, 4921,
       4921,  134, 3980, 1142, 1304,  134, 4631,  134, 2730, 1165, 4174,
       1086, 4001, 3985, 1906, 4081, 3418, 4710, 1070, 3534, 2749, 1260,
       2761, 1327, 3143, 2260, 2809, 3879, 4650], dtype=int32)

In [ ]:
## Creating model
n_timesteps = 100
embedding_vector_features=50
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(1000, return_sequences=True), input_shape=(n_timesteps, 1))) ##Just add bidirectional!!, except it would just behave as normal LSTM Model
model1.add(Dropout(0.3))
model1.add(TimeDistributed(Dense(1, activation='sigmoid')))
model1.compile(loss='binary_crossentropy',optimizer='adamax',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 40, 50)            250000    
                                                                 
 bidirectional_2 (Bidirectio  (None, 40, 2000)         8408000   
 nal)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 40, 2000)          0         
                                                                 
 time_distributed (TimeDistr  (None, 40, 1)            2001      
 ibuted)                                                         
                                                                 
Total params: 8,660,001
Trainable params: 8,660,001
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
len(embedded_docs),y.shape

(15185, (15185,))

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final[1]

array([ 225, 4048, 3320,  119, 1016, 2686, 1770, 3480, 3119, 3479,   37,
        256, 2329, 4394, 2123, 3275, 4435, 2123, 3029, 1808, 3342, 3549,
       4331, 3985, 3951,  875, 4090, 2748, 4174, 2134, 3951, 3985,  241,
       3366,  534, 4261, 1980, 4174, 1945, 3182], dtype=int32)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.25, random_state=32)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.25, random_state=32)

In [ ]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=30,batch_size=64)

Epoch 1/30
178/178 [==============================] - 687s 4s/step - loss: 0.1950 - accuracy: 0.9536 - val_loss: 0.2167 - val_accuracy: 0.9442
Epoch 2/30
178/178 [==============================] - 676s 4s/step - loss: 0.1847 - accuracy: 0.9561 - val_loss: 0.2161 - val_accuracy: 0.9442
Epoch 3/30
178/178 [==============================] - 674s 4s/step - loss: 0.1860 - accuracy: 0.9561 - val_loss: 0.2160 - val_accuracy: 0.9442
Epoch 4/30
178/178 [==============================] - 672s 4s/step - loss: 0.1844 - accuracy: 0.9561 - val_loss: 0.2162 - val_accuracy: 0.9442
Epoch 5/30
178/178 [==============================] - 673s 4s/step - loss: 0.1842 - accuracy: 0.9561 - val_loss: 0.2156 - val_accuracy: 0.9442
Epoch 6/30
178/178 [==============================] - 673s 4s/step - loss: 0.1822 - accuracy: 0.9561 - val_loss: 0.2204 - val_accuracy: 0.9442
Epoch 7/30
178/178 [==============================] - 673s 4s/step - loss: 0.1839 - accuracy: 0.9561 - val_loss: 0.2157 - val_accuracy: 0.9442

In [ ]:
y_pred=model1.predict(X_test)
classes_y = np.argmax(y_pred,axis=1)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, classes_y)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,classes_y)

0.18883328943903083

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,classes_y))

In [ ]:
model1.save_weights("./model.h5")

In [ ]:
model1 = load_model('model_keras/model.h5')

In [ ]:
model1 = tf.keras.models.load_model('model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

# SpaCy

In [ ]:
dataread = pd.read_csv('./fake_job_postings.csv')

In [ ]:
import spacy

In [ ]:
nlp = spacy.blank("en")

textcat = nlp.create_pipe(
              "textcat",
              config={
                "exclusive_classes": True,
                "architecture": "bow"})

nlp.add_pipe(textcat)

In [ ]:
dataread().head

check missing data

In [ ]:
missingno.matrix(data)

In [ ]:
print(dataread.columns)
data.describe()

In [ ]:
import seaborn as sns

In [ ]:
sns.countplot(dataread.fraudulent)
dataread.groupby('fraudulent').count()['title'].reset_index().sort_values(by='title',ascending=False)

In [ ]:
columns=['job_id', 'telecommuting', 'has_company_logo', 'has_questions', 'salary_range', 'employment_type']
for col in columns:
    del data[col]

dataread.fillna(' ', inplace=True)

In [ ]:
dataread.head()

In [ ]:
def split(location):
    l = location.split(',')
    return l[0]

dataread['title'] = dataread.location.apply(split)

In [ ]:
titlejob = dict(dataread.titlejob.value_counts()[:11])
del titlejob[' ']
plt.figure(figsize=(8,6))
plt.title('No. of job postings title wise', size=20)
plt.bar(titlejob.keys(), titlejob.values())
plt.ylabel('No. of jobs', size=10)
plt.xlabel('title', size=10)

In [ ]:
experience = dict(dataread.required_experience.value_counts())
del experience[' ']
plt.bar(experience.keys(), experience.values())
plt.xlabel('Experience', size=10)
plt.ylabel('no. of jobs', size=10)
plt.xticks(rotation=35)
plt.show()

In [ ]:
print(data.title.value_counts()[:10])

In [ ]:
dataread['text']=dataread['title']+' '+dataread['location']+' '+dataread['company_profile']+' '+dataread['description']+' '+dataread['requirements']+' '+dataread['benefits']
del dataread['title']
del dataread['location']
del dataread['department']
del dataread['company_profile']
del dataread['description']
del dataread['requirements']
del dataread['benefits']
del dataread['required_experience']
del dataread['required_education']
del dataread['industry']
del dataread['function']
del dataread['country']

In [ ]:
dataread.head()

In [ ]:
fraudjobs_text = dataread[dataread.fraudulent==1].text
actualjobs_text = dataread[dataread.fraudulent==0].text

In [ ]:
STOPWORDS = spacy.lang.en.stop_words.STOP_WORDS
plt.figure(figsize = (16,14))
wc = WordCloud(min_font_size = 3,  max_words = 3000 , width = 1600 , height = 800 , stopwords = STOPWORDS).generate(str(" ".join(fraudjobs_text)))
plt.imshow(wc,interpolation = 'bilinear')

In [ ]:
plt.figure(figsize = (16,14))
wc = WordCloud(min_font_size = 3,  max_words = 3000 , width = 1600 , height = 800 , stopwords = STOPWORDS).generate(str(" ".join(actualjobs_text)))
plt.imshow(wc,interpolation = 'bilinear')

Cleaning Data

In [ ]:
punctuations = string.punctuation

nlp = spacy.load('en')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

parser = English()

In [ ]:
def spacy_tokenizer(sentence):
  mytokens = parser(sentence)
  mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
  mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

  return mytokens

In [ ]:
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

def clean_text(text):
    return text.strip().lower()

In [ ]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,3))

In [ ]:
Xb_train, Xb_test, yb_train, yb_test = train_test_split(data.text, data.fraudulent, test_size=0.3)

In [ ]:
clf = LogisticRegression()

# Create pipeline using Bag of Words
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', clf)])

# fitting our model.
pipe.fit(Xb_train,yb_train)

In [ ]:
# Predicting with a test dataset
predicted = pipe.predict(Xb_test)

# Model Accuracy
print("Logistic Regression Accuracy:", accuracy_score(yb_test, predicted))
print("Logistic Regression Recall:", recall_score(yb_test, predicted))

In [ ]:
plot_confusion_matrix(pipe, Xb_test, yb_test, cmap='Blues', values_format=' ')

# Bikin Prediksi

In [ ]:
textcat.add_label("fraud")
textcat.add_label("real")

In [ ]:
texts = ["HAVE YOU EVER BEEN A DJ????? It's gonna be wild",
         "URGENT Reply to this message for applying as a dj!!" ]
docs = [nlp.tokenizer(text) for text in texts]

In [ ]:
# Use textcat to get the scores for each doc
textcat = nlp.get_pipe('textcat')
scores, _ = textcat.predict(docs)

print(scores)

In [ ]:
# From the scores, find the label with the highest score/probability
predicted_labels = scores.argmax(axis=1)
print([textcat.labels[label] for label in predicted_labels])